## An Object-Oriented Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- starter(): Returns an initial message to the user (Resulting from instance_starter prompt)
- response_for(user_says): Returns an assistance response to user_says

In [ ]:
import uuid
import time
from chatbot_db import Chatbot

#### Create a chatbot "Coach" for user X

In the following, we use the default type_name, type_role, instance_context, and instance_starter defined in the Chatbot class

In [ ]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="053e97a0-6a91-4589-8602-340aa47b6376",
    user_id="7515865e-4097-4dd7-9567-d3c7a4c1ed07",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

Retrieve the complete conversation (held so far) or the bot information (type_role, instance_context, and instance_starter) as retrieved from the database. Both may be used to display that on a frontend.

In [ ]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="053e97a0-6a91-4589-8602-340aa47b6376",
    user_id="7515865e-4097-4dd7-9567-d3c7a4c1ed07"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

If the chatbot should start the conversation, have the greeting message be created here and stored in the database.

In [ ]:
print(bot.starter())

URL to be handed out to the user

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Create another chatbot "Learning Assistant" for user Y
In the following, we provide our own type_name, type_role, instance_context, and instance_starter.
By providing a number we can automatically create a set amount of users.

In [ ]:
number = 40 # amount users to be created
# Change to list with hardcoded value, if you want to use and existing user.
uuids = [str(uuid.uuid4()) for _ in range(number)]
c  = 0 # counter for successful requests don't change
error_c = 0 # counter for failed requests, don't change
for id in uuids:
    learning_assistant = Chatbot(
        database_file="data/chatbot.db", 
        type_id="c8db2547-25b5-43b5-b6b0-20c377b543fc",
        user_id=id,
        type_name="Learning Assistant",
        type_role='''
        Your are a mindful learning assistant. You always reply with short responses.
        You help users when they do their homework for school or prepare for exams.
        ''',
        instance_context='''
        You are now having a conversation with Kevin.
        Kevin is preparing for a math exam. The topic of examination is the pythagoras theorem.
        ''',
        instance_starter='''
            Create a short message to greet Kevin.
        '''
    )
    try:
        print(learning_assistant.starter())
        
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


In [ ]:
pythonanywhere_username = "<ENTER YOUR PYTHONANYWHERE USERNAME HERE>"
learning_assistant.get_bot_urls(pythonanywhere_username)

In [ ]:
pythonanywhere_username = "heej"
database_file="data/chatbot.db"
connection = sqlite3.connect(database_file)
cursor = connection.cursor()
cursor.execute("SELECT type, user FROM chatbot_instances")
rows = cursor.fetchall()
for row in rows:
    print("https://{}pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, row[0], row[1]))
    
cursor.close()
connection.close()